In [1]:
import pandas as pd
import numpy as np
import os
from time import time
import socket
from pathlib import Path
import unicodedata
import unidecode
import sys

import warnings
warnings.filterwarnings('ignore')

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from py4j.java_gateway import java_import
from functools import reduce
from pyspark.sql import DataFrame
from pyspark import SparkContext

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem.snowball import SnowballStemmer

# 0. Init

In [3]:
memory = '10g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [4]:
try:
    spark
except NameError:
    print('Create Local SparkSession')
    spark=SparkSession.builder.config("spark.driver.host", "localhost").appName("extract-timelines").getOrCreate()
    
# IgnoreCorruptFiles
spark.conf.set("spark.sql.files.ignoreCorruptFiles", "true")
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

sc = spark.sparkContext

Create Local SparkSession


In [5]:
country_code="FR"
language_code="fr"
print('Country:',country_code)
print('Language:',language_code)

Country: FR
Language: fr


In [6]:
language_dict = {'fr' : 'French',
                'en' : 'English',
                'es' : 'Spanish',
                'pt' : 'Portuguese'}

# 1. Import tweets

## 1.1. Path to data

In [7]:
# Paths to data
path_to_data = "../data/"
path_to_parquets = os.path.join(path_to_data,'chunks','IDF-departments-to-analyze')
parquet_files = sorted([x for x in Path(path_to_parquets).glob("**/*.parquet")])
path_to_external_data = os.path.join(path_to_data, "external-data/")
path_to_output = os.path.join(path_to_data,'visualisation_data','tweets_symptoms_medra')

In [8]:
print('List files to be processed...')

fs=spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
list_status=fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(path_to_parquets))

paths=[file.getPath().toString() for file in list_status]
np.random.seed(0)
paths=np.random.permutation(sorted(parquet_files))

print('# Files:', len(paths))

List files to be processed...
# Files: 16


## 1.2. Read tweets

In [9]:
tweets=spark.read.option("encoding", "UTF-8").parquet(path_to_parquets)
#tweets=spark.read.option("encoding", "UTF-8").parquet(path_to_parquets+"/0.parquet")

In [10]:
print("Number of tweets : %d" % tweets.count())
print("Number of unique users : %d" % tweets.select('user_id').distinct().count())

Number of tweets : 29647175
Number of unique users : 30651


In [11]:
#tweets=tweets.filter(tweets.user_id=='1000008320369950720')

## 1.3. Clean tweets

We keep only useful columns, and remove the accents and the punctuation. We stem all the words in the text.

In [12]:
# UNIDECODE : functions to remove accents
def make_trans():
    matching_string = ""
    replace_string = ""

    for i in range(ord(" "), sys.maxunicode):
        name = unicodedata.name(chr(i), "")
        if "WITH" in name:
            try:
                base = unicodedata.lookup(name.split(" WITH")[0])
                matching_string += chr(i)
                replace_string += base
            except KeyError:
                pass

    return matching_string, replace_string

def clean_text(c):
    matching_string, replace_string = make_trans()
    return F.translate(
        F.regexp_replace(c, "\p{M}", ""), 
        matching_string, replace_string
    ).alias(c)

In [13]:
# Preprocess tweets
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    #tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


In [14]:
def clean_dataset(df):
    
    df=df.select(
            'user_id',
            F.date_format(F.col('created_at'),"yyyy-MM-dd").alias('day').cast("date"),
            #'year','month','day',
            F.lower(F.col('full_text')).alias('text'),
            #F.lower(F.col('text')),
            'lang'
            )
    
#     df=df.withColumn("merge", concat_ws("-", $"year", $"month", $"day"))\
#         .withColumn("day", to_date(unix_timestamp($"merge", "yyyy-MM-dd").cast("date")))\
#         .drop("merge")

    df = df.repartition(160)

    # remove retweets
    df = df.filter(~ df.text.startswith('rt'))
   
    # identify userid, url, etc.
    pre_process_udf = F.udf(lambda x:text_processor.pre_process_doc(x))
    df=df.withColumn('text', pre_process_udf('text'))
    
    # language : french
    df = df.filter(df.lang==language_code)
    
    return df

In [15]:
tweets = clean_dataset(tweets)

# Clean text: remove accents
tweets = tweets.select('user_id', 'day', 'text', clean_text('text').alias('clean_text'))
# Cleaning: remove and punctuation-smileys
tweets = tweets.withColumn('clean_text', F.regexp_replace('clean_text', '[^\sa-zA-Z0-9@#<>]', ' '))

In [16]:
print(tweets.filter(tweets.clean_text.startswith('je')).count())

214959


In [21]:
tweets.show(5)

+-------------------+----------+--------------------+--------------------+
|            user_id|       day|                text|          clean_text|
+-------------------+----------+--------------------+--------------------+
|1036925143036121092|2019-12-01|<user> comme qui ...|<user> comme qui ...|
|1001101575643910144|2020-04-29|juste pour vous r...|juste pour vous r...|
|1000040081527312386|2020-01-14|<user> au repos j...|<user> au repos j...|
|1012688936081248257|2020-04-19|<user> <user> pas...|<user> <user> pas...|
|         1042047907|2020-04-06|    réellement <url>|    reellement <url>|
+-------------------+----------+--------------------+--------------------+
only showing top 5 rows



In [16]:
#Stemming

for col in ['text','clean_text'] :

    # Tokenize
    tokenizer=TextPreProcessor(tokenizer=SocialTokenizer(lowercase=True).tokenize,
                            dicts=[emoticons])
    tokenize_udf = F.udf(lambda x:tokenizer.pre_process_doc(x), ArrayType(StringType()))
    tweets=tweets.withColumn('tokens_'+col, tokenize_udf(col))
    #tokenizer = Tokenizer(inputCol=col, outputCol='tokens_'+col)
    #tweets = tokenizer.transform(tweets)

    # Stem tokens
    stemmer = SnowballStemmer(language=language_dict.get(language_code).lower())
    stemmer_udf = F.udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
    tweets = tweets.withColumn("tokens_stem_"+col, stemmer_udf('tokens_'+col))

    # Join stemmed tokens
    join_udf = F.udf(lambda x: " ".join(x), StringType())
    tweets = tweets.withColumn('stemmed_'+col, join_udf(F.col("tokens_stem_"+col)))
     #tweets = tweets.withColumn("stemmed_text", F.concat_ws(" ", "tokens_stem"))
tweets = tweets.select('user_id','day','text','clean_text','stemmed_text','stemmed_clean_text')

Reading english - 1grams ...
Reading english - 1grams ...


In [17]:
#tweets.show(truncate=False)

# 2. Symptoms analysis based on MedRa dictionary 

In [18]:
medra=spark.read.format('csv').option('header','true').load(os.path.join(path_to_external_data,'MedRa','medra_lang','medra_'+str(language_code)+'.csv'))

In [19]:
def count_number_symptoms(df,version):
    
    print('Version : ' + str(version))
    
    symptoms = [row.pt_name for row in medra.select('pt_name').distinct().collect()]
    all_llt_cols = [row.llt_name_as_col for row in medra.select('llt_name_as_col').distinct().collect()]
    all_pt_codes = [row.pt_code for row in medra.select('pt_code').distinct().collect()]
    v = versions.get(version)
        
    for symptom in symptoms :
        targets = [row[v+'llt_name'] for row in medra.filter(medra['pt_name']==symptom).select(v+'llt_name').collect()]
        targets_col = [row['llt_name_as_col'] for row in medra.filter(medra['pt_name']==symptom).select('llt_name_as_col').collect()]
        symptom_code = medra.filter(medra['pt_name']==symptom).select('pt_code').collect()[0]['pt_code']
        
        for target in targets:
            target_col_name=medra.filter(medra[v+'llt_name']==target).select('llt_name_as_col').collect()[0]['llt_name_as_col']
            df=df.withColumn('llt_'+target_col_name, F.col(v+'text').contains(target).cast('int'))
            
        #df=df.withColumn(symptom, F.col(v+'text').rlike("|".join(targets)).cast('int'))
        df=df.withColumn('symptom_counts', sum(df['llt_'+c].cast('long') for c in targets_col))
        df=df.withColumn('pt_'+symptom, F.col('symptom_counts')>=1)
        df=df.withColumn('pt_'+symptom, F.col('pt_'+symptom).cast('int'))
        df=df.withColumn('pt_code_'+symptom_code, F.col('pt_'+symptom))


    df = df.withColumn('nb_symptoms', sum(df['pt_'+c].cast('long') for c in symptoms))
    df = df.withColumn('contains_symptom', F.col('nb_symptoms')>=1)
    df = df.withColumn('contains_symptom', F.col('contains_symptom').cast('int'))
    
    
    # groupbyday
    #df = df.filter(df.contains_symptom==1)
    cols_symptoms_to_keep = ['contains_symptom']+['llt_' + x for x in all_llt_cols]+['pt_'+x for x in symptoms]+['pt_code_'+x for x in all_pt_codes]
    df_symptoms = df.select(['day']+cols_symptoms_to_keep)\
            .groupby('day')\
            .agg(*[F.sum(x).alias(x) for x in cols_symptoms_to_keep], F.count('*').alias('nb_tweets'))\
            .orderBy('day')
    
    return df_symptoms

In [ ]:
versions = {'V0' : '',
           'V1' : 'clean_',
           'V2' : 'stemmed_',
           'V3' : 'stemmed_clean_'}

for version in versions.keys():
    tweets_symptoms = count_number_symptoms(tweets, version)
    tweets_symptoms = tweets_symptoms.withColumn('day', F.to_timestamp(tweets_symptoms.day, 'yyyy-MM-dd')).toPandas()
    tweets_symptoms.to_csv(os.path.join(path_to_output,'tweets_symptoms_'+str(country_code)+'_'+str(version)+'.csv'), header=True)

Version : V0


In [ ]:
# def count_number_symptoms(df,version):
    
#     print('Version : ' + str(version))
    
#     symptoms = [row.pt_name for row in medra.select('pt_name').distinct().collect()]
#     v = versions.get(version)
        
#     for symptom in symptoms :
#         targets = [row[v+'llt_name'] for row in medra.filter(medra['pt_name']==symptom).select(v+'llt_name').collect()]
#         df=df.withColumn(symptom, F.col(v+'text').rlike("|".join(targets)).cast('int'))

#     df = df.withColumn('nb_symptoms', sum(df[c].cast('long') for c in symptoms))
#     df = df.withColumn('contains_symptom', F.col('nb_symptoms')>=1)
#     df = df.withColumn('contains_symptom', F.col('contains_symptom').cast('int'))
    
    
#     # groupbyday
#     df = df.filter(df.contains_symptom==1)
#     df_symptoms = df.select(['day','contains_symptom'])\
#             .groupby('day').sum()\
#             .orderBy('day')\
#             .toDF(*['day', 'contains_symptom_'+str(version)])
    
#     return df_symptoms

In [ ]:
# versions = {'V0' : '',
#            'V1' : 'clean_',
#            'V2' : 'stemmed_',
#            'V3' : 'stemmed_clean_'}

# for version in versions.keys():
#     if version=='V0':
#         tweets_symptoms = count_number_symptoms(tweets, version)
#     else:
#         df = count_number_symptoms(tweets, version)
#         tweets_symptoms = tweets_symptoms.join(df, on=['day'], how='full_outer')

In [ ]:
# Export table to csv
# tweets_symptoms = tweets_symptoms.withColumn('day', F.to_timestamp(tweets_symptoms.day, 'yyyy-MM-dd')).toPandas()
# tweets_symptoms.to_csv(os.path.join(path_to_output,'tweets_symptoms_'+str(country_code)+'.csv'), header=True)